In [46]:
import open3d as o3d
import numpy as np

In [ ]:
class Gripper:
    def __init__(self, closure_volume_size: list, finger_thickness: float = 0.01):
        self._closure_volume_size = np.array(closure_volume_size)
        self._transforms = self._create_transforms(finger_size=[finger_thickness, self._closure_volume_size[1], self._closure_volume_size[2]], palm_size=[self._closure_volume_size[0], self._closure_volume_size[1], finger_thickness])
        self._meshes = self._create_meshes(finger_size=[finger_thickness, self._closure_volume_size[1], self._closure_volume_size[2]], palm_size=[self._closure_volume_size[0], self._closure_volume_size[1], finger_thickness])
        self._bboxes = self._create_bboxes(closure_volume_size=self._closure_volume_size, transforms=self._transforms)
    
    def _create_bboxes(self, closure_volume_size, transforms):
        bboxes = {}
        # closure volume bounding box
        min_bound = transforms['base_to_closure_volume_bottom'][:3, 3]
        max_bound = min_bound + closure_volume_size
        bboxes['closure_volume'] =o3d.geometry.AxisAlignedBoundingBox(min_bound, max_bound)
        return bboxes

    def _create_transforms(self, finger_size, palm_size):
        transforms = {}
        transforms['base_to_palm_bottom'] = np.eye(4)
        transforms['base_to_right_finger_bottom'] = np.eye(4)
        transforms['base_to_left_finger_bottom'] = np.eye(4)
        transforms['base_to_closure_volume_bottom'] = np.eye(4)

        # Palm transform
        transforms['base_to_palm_bottom'][0, 3] = -palm_size[0] / 2
        transforms['base_to_palm_bottom'][1, 3] = -palm_size[1] / 2
        transforms['base_to_palm_bottom'][2, 3] = -finger_size[2] - palm_size[2]
        transforms['base_to_palm_top'] = transforms['base_to_palm_bottom'].copy()
        transforms['base_to_palm_top'][2, 3] += palm_size[2]
        # Right finger transform
        transforms['base_to_right_finger_bottom'][0, 3] = palm_size[0] / 2
        transforms['base_to_right_finger_bottom'][1, 3] = -palm_size[1] / 2
        transforms['base_to_right_finger_bottom'][2, 3] = -finger_size[2]
        transforms['base_to_right_finger_top'] = transforms['base_to_right_finger_bottom'].copy()
        transforms['base_to_right_finger_top'][2, 3] += finger_size[2]
        # Left finger transform
        transforms['base_to_left_finger_bottom'][0, 3] = -palm_size[0] / 2 - finger_size[0]
        transforms['base_to_left_finger_bottom'][1, 3] = -palm_size[1] / 2
        transforms['base_to_left_finger_bottom'][2, 3] = -finger_size[2]
        transforms['base_to_left_finger_top'] = transforms['base_to_left_finger_bottom'].copy()
        transforms['base_to_left_finger_top'][2, 3] += finger_size[2]
        # Closure volume transform
        transforms['base_to_closure_volume_bottom'][0, 3] = -palm_size[0] / 2 + finger_size[0]
        transforms['base_to_closure_volume_bottom'][1, 3] = -palm_size[1]
        transforms['base_to_closure_volume_bottom'][2, 3] = -finger_size[2]
        return transforms
    
    def has_collision(self, points: o3d.geometry.PointCloud) -> bool:
        """
        Check if the given points collide with the gripper's palm or fingers.
        """
        collision_check_meshes = [self._meshes['palm'], self._meshes['right_finger'], self._meshes['left_finger']]
        for bbox in collision_check_meshes.get_axis_aligned_bounding_box():
            # crop the point cloud to the bounding box
            cropped_points = points.crop(bbox)
            if len(cropped_points.points) > 0:
                return True
        return False
    
    def has_point_in_closure_volume(self, points: o3d.geometry.PointCloud) -> bool:
        """
        Check if the given points are inside the closure volume of the gripper.
        """
        cropped_points = points.crop(self._bboxes['closure_volume'])
        return len(cropped_points.points) > 0

    def get_maximum_movable_distance(self, points: o3d.geometry.PointCloud,  finger_size, palm_size, transforms):
        """
        Calculate the maximum distance that the gripper can move along z-axis without collision.
        """
        # get the bounding box of the point cloud and use its how far it extends along the z-axis
        max_extend_distance = points.get_axis_aligned_bounding_box().get_extent()[2]
        # create a extended bounding boxes against right finger, left finger, palm
        extended_right_finger_bbox = o3d.geometry.AxisAlignedBoundingBox(
            min_bound=transforms['base_to_right_finger_top'][:3, 3],
            max_bound=transforms['base_to_right_finger_top'][:3, 3] + np.array([finger_size[0], finger_size[1], max_extend_distance])
        )
        extended_left_finger_bbox = o3d.geometry.AxisAlignedBoundingBox(
            min_bound=transforms['base_to_left_finger_top'][:3, 3],
            max_bound=transforms['base_to_left_finger_top'][:3, 3] + np.array([finger_size[0], finger_size[1], max_extend_distance])
        )
        extended_palm_bbox = o3d.geometry.AxisAlignedBoundingBox(
            min_bound=transforms['base_to_palm_top'][:3, 3],
            max_bound=transforms['base_to_palm_top'][:3, 3] + np.array([palm_size[0], palm_size[1], max_extend_distance + finger_size[2]])
        )
        # find the nearest point in z-axis in the cropped point cloud
        nearest_z_point = None
        collided_part_name = None
        for bbox, name in zip([extended_right_finger_bbox, extended_left_finger_bbox, extended_palm_bbox], ['right_finger', 'left_finger', 'palm']):
            cropped_points = points.crop(bbox)
            if len(cropped_points.points) > 0:
                z_coords = np.array(cropped_points.points)[:, 2]
                if nearest_z_point is None or np.min(z_coords) < nearest_z_point:
                    nearest_z_point = np.min(z_coords)
                    collided_part_name = name
        if nearest_z_point is not None:
            if collided_part_name == 'right_finger' or collided_part_name == 'left_finger':
                return nearest_z_point # finger top is the same as the origin, so we can return the nearest z point directly
            elif collided_part_name == 'palm':
                return nearest_z_point + finger_size[2] # palm top is located at the - finger size, so we need to add it to the nearest z point
        else:
            return None
    
    def crop_by_closure_volume(self, points: o3d.geometry.PointCloud) -> o3d.geometry.PointCloud:
        """
        Crop the given point cloud by the closure volume of the gripper.
        """
        return points.crop(self._bboxes['closure_volume'])

    def _create_meshes(self, finger_size, palm_size):
        meshes = {}
        # Right finger mesh
        right_finger_mesh = o3d.geometry.TriangleMesh.create_box(finger_size[0], finger_size[1], finger_size[2], True)
        right_finger_mesh.translate(self._transforms['base_to_right_finger_bottom'][:3, 3])
        right_finger_mesh.paint_uniform_color([0.1, 0.4, 0.8])
        meshes['right_finger'] = right_finger_mesh
        # Left finger mesh
        left_finger_mesh = o3d.geometry.TriangleMesh.create_box(finger_size[0], finger_size[1], finger_size[2])
        left_finger_mesh.translate(self._transforms['base_to_left_finger_bottom'][:3, 3])
        left_finger_mesh.paint_uniform_color([0.5, 0.1, 0.6])
        meshes['left_finger'] = left_finger_mesh
        # Palm mesh
        palm_mesh = o3d.geometry.TriangleMesh.create_box(palm_size[0], palm_size[1], palm_size[2])
        palm_mesh.translate(self._transforms['base_to_palm_bottom'][:3, 3])
        palm_mesh.paint_uniform_color([0.4, 0.1, 0.3])
        meshes['palm'] = palm_mesh
        # Optionally, base frame mesh
        frame_mesh = o3d.geometry.TriangleMesh.create_coordinate_frame()
        frame_mesh = frame_mesh.scale(0.1, center=[0,0,0])
        meshes['frame'] = frame_mesh
        return meshes

    @property
    def meshes(self):
        return list(self._meshes.values())


In [50]:
gripper = Gripper([0.05, 0.03, 0.05], 0.01)
meshes = gripper.meshes
o3d.visualization.draw_geometries(meshes)

In [ ]:
meshes[0].get_axis_aligned_bounding_box().get_center()  # This line is just to ensure the code runs without error